In [4]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from rsome import ro 
import rsome as rso                           # import the ro module
from rsome import grb_solver as grb
from torch.utils.data import DataLoader, TensorDataset
import sbibm
import model

In [5]:
import networkx as nx

G = nx.grid_2d_graph(100, 100).to_directed()
A = nx.incidence_matrix(G, oriented=True).todense()

In [6]:
weights = {e : np.random.random() for e in G.edges}
nx.set_edge_attributes(G, weights, name = 'weight')

In [7]:
b = np.zeros(len(G.nodes)) # b entries: 1 for source, -1 for target, 0 o.w.
b[0] = -1
b[-1] = 1 

In [8]:
model = ro.Model()

w = model.dvar(len(G.edges))
c = np.array(list(nx.get_edge_attributes(G, name = 'weight').values()))

model.min(c @ w)
model.st(w <= 1)
model.st(w >= 0)
model.st(A @ w == b)

model.solve(grb)

KeyboardInterrupt: 

In [ ]:
print(f"robust_path_length = {model.get()}")

nx_path_length = 2.7237250949518312
lp_path_length = 8.0
